# CSE547 - Colab 2
## Frequent Pattern Mining in Spark

Adapted from Stanford CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 53kB/s 
     |████████████████████████████████| 204kB 40.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=1eb39717feabf08c0610d2af056a1066bca0d5bd15e4033eb311a31191f7e309
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

Let's initialize the Spark context.

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-16 19:42:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.22.117.137, 34.196.154.11, 52.200.114.213, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.22.117.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.0MB/s    in 0.9s    

2020-04-16 19:42:36 (14.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://fcf84b2c.ngrok.io


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [0]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [9]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [82]:
x = products.filter(products.product_id == 44910).select(products.product_name).rdd.map(tuple)
x.take(1)

[('Organic Spring Mix Salad',)]

In [10]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [14]:
# YOUR CODE HERE
orders.registerTempTable("orders")
products.registerTempTable("products")

query = """SELECT o.*, p.product_name as product_name
          FROM orders o left join products p
          on o.product_id = p.product_id"""
orders_joined = spark.sql(query)
orders_joined.show(5)

+--------+----------+-----------------+---------+--------------------+
|order_id|product_id|add_to_cart_order|reordered|        product_name|
+--------+----------+-----------------+---------+--------------------+
|       1|     49302|                1|        1|    Bulgarian Yogurt|
|       1|     11109|                2|        1|Organic 4% Milk F...|
|       1|     10246|                3|        0|Organic Celery He...|
|       1|     49683|                4|        0|      Cucumber Kirby|
|       1|     43633|                5|        1|Lightly Smoked Sa...|
+--------+----------+-----------------+---------+--------------------+
only showing top 5 rows



In [62]:
from pyspark.sql.functions import col, isnan, when, trim

def to_null(c):
    return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == "")), col(c))


orders_clean = orders_joined.select([to_null(c).alias(c) for c in orders_joined.columns]).na.drop()
orders_clean.show()

+--------+----------+-----------------+---------+--------------------+
|order_id|product_id|add_to_cart_order|reordered|        product_name|
+--------+----------+-----------------+---------+--------------------+
|       1|     49302|                1|        1|    Bulgarian Yogurt|
|       1|     11109|                2|        1|Organic 4% Milk F...|
|       1|     10246|                3|        0|Organic Celery He...|
|       1|     49683|                4|        0|      Cucumber Kirby|
|       1|     43633|                5|        1|Lightly Smoked Sa...|
|       1|     13176|                6|        0|Bag of Organic Ba...|
|       1|     47209|                7|        0|Organic Hass Avocado|
|       1|     22035|                8|        1|Organic Whole Str...|
|      36|     39612|                1|        0|Grated Pecorino R...|
|      36|     19660|                2|        1|        Spring Water|
|      36|     49235|                3|        0| Organic Half & Half|
|     

In [115]:
import pyspark.sql.functions as f
orders_basket = orders_clean.groupby("order_id").agg(f.collect_list(orders_clean.product_name))
orders_basket = orders_basket.select(col("order_id").alias("order_id"), col("collect_list(product_name)").alias("product_name"))
orders_basket.show(5)

+--------+--------------------+
|order_id|        product_name|
+--------+--------------------+
|    1342|[Bag of Organic B...|
|    1591|[Cracked Wheat, O...|
|    4519|[Beet Apple Carro...|
|    4935|             [Vodka]|
|    6357|[Fresh Mozzarella...|
+--------+--------------------+
only showing top 5 rows



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [0]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="product_name", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(orders_basket)

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
#model.transform(df).show()

Compute how many Frequent Itemsets and Association Rules were generated by running FP-growth.


In [117]:
# YOUR CODE HERE
model.freqItemsets.count()

120

In [118]:
model.associationRules.show()

+----------+----------+----------+----+
|antecedent|consequent|confidence|lift|
+----------+----------+----------+----+
+----------+----------+----------+----+



Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many Frequent Itemsets and Association Rules were generated.

In [0]:
# YOUR CODE HERE


fpGrowth = FPGrowth(itemsCol="product_name", minSupport=0.001, minConfidence=0.5)
model_2 = fpGrowth.fit(orders_basket)


In [128]:
x = model_2.freqItemsets
x.orderBy(x["freq"].desc()).show(21, False)

+------------------------+-----+
|items                   |freq |
+------------------------+-----+
|[Banana]                |18726|
|[Bag of Organic Bananas]|15480|
|[Organic Strawberries]  |10894|
|[Organic Baby Spinach]  |9784 |
|[Large Lemon]           |8135 |
|[Organic Avocado]       |7409 |
|[Organic Hass Avocado]  |7293 |
|[Strawberries]          |6494 |
|[Limes]                 |6033 |
|[Organic Raspberries]   |5546 |
|[Organic Blueberries]   |4966 |
|[Organic Whole Milk]    |4908 |
|[Organic Cucumber]      |4613 |
|[Organic Zucchini]      |4589 |
|[Organic Yellow Onion]  |4290 |
|[Organic Garlic]        |4158 |
|[Seedless Red Grapes]   |4059 |
|[Asparagus]             |3868 |
|[Organic Grape Tomatoes]|3823 |
|[Organic Red Onion]     |3818 |
|[Yellow Onions]         |3762 |
+------------------------+-----+
only showing top 21 rows



In [126]:
model_2.associationRules.count()

11

In [129]:
y = model_2.associationRules
y.show(20, False)

+-----------------------------------------------------------------+------------------------+------------------+------------------+
|antecedent                                                       |consequent              |confidence        |lift              |
+-----------------------------------------------------------------+------------------------+------------------+------------------+
|[Organic Kiwi, Organic Hass Avocado]                             |[Bag of Organic Bananas]|0.5459770114942529|4.627719489738336 |
|[Organic Raspberries, Organic Hass Avocado, Organic Strawberries]|[Bag of Organic Bananas]|0.5984251968503937|5.072272070642333 |
|[Organic Broccoli, Organic Hass Avocado]                         |[Bag of Organic Bananas]|0.5048231511254019|4.278897986822536 |
|[Organic Unsweetened Almond Milk, Organic Hass Avocado]          |[Bag of Organic Bananas]|0.5141065830721003|4.357584667849303 |
|[Yellow Onions, Strawberries]                                    |[Banana]        

Now, for 
```minsupport=0.001```, 
visualize the top Frequent Itemsets and Association Rules.

In addition for
```minsupport=0.001```, 
calculate the "interest" of the Association Rules. 

(Hint: One way to do this by joining the Frequent Itemsets with the Association Rules)




In [130]:
orders_basket.count()

131209

In [0]:
x.registerTempTable("frequent")
y.registerTempTable("association")

query = """select r.antecedent, r.consequent, r.confidence, r.lift, (r.confidence - r.proportion) as interest
            from(
            select a.*, f.freq as freq, (f.freq/131209) as proportion
            from association a join frequent f
            on a.consequent = f.items) as r"""
z = spark.sql(query)

In [135]:
z.orderBy(z['interest'].desc()).show(5, False)

+-----------------------------------------------------------------+------------------------+------------------+-----------------+-------------------+
|antecedent                                                       |consequent              |confidence        |lift             |interest           |
+-----------------------------------------------------------------+------------------------+------------------+-----------------+-------------------+
|[Organic Raspberries, Organic Hass Avocado, Organic Strawberries]|[Bag of Organic Bananas]|0.5984251968503937|5.072272070642333|0.48044548509281615|
|[Organic Cucumber, Organic Hass Avocado, Organic Strawberries]   |[Bag of Organic Bananas]|0.546875          |4.635330870478036|0.4288952882424224 |
|[Organic Kiwi, Organic Hass Avocado]                             |[Bag of Organic Bananas]|0.5459770114942529|4.627719489738336|0.42799729973667533|
|[Organic Navel Orange, Organic Raspberries]                      |[Bag of Organic Bananas]|0.541218

In [137]:
s = 'Organic Raspberries, Organic Hass Avocado, Organic Strawberries'  # Taken from the Q1 example
print(', '.join(sorted(s.split(', '))))

Organic Hass Avocado, Organic Raspberries, Organic Strawberries


To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.

Once you obtained the desired results, **submit your solution for this Colab**!